In [4]:
%reload_ext autoreload

from config.Neo_connect import Neo4jConnection
import pandas as pd
import config.db_settings as db_config

# stablish connection with neo4j
conn = Neo4jConnection(uri="bolt://localhost:7687", user=db_config.USER, pwd=db_config.PASSWORD)
#result = conn.query('create database {}'.format(DATABASE))



In [5]:
#create nodes

#-----------------------------------
node_queries = [
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/stock_data.csv' as row FIELDTERMINATOR ',' CREATE (:stock {symbol:row.symbol,lastPrice :row.lastPrice,
dayHigh: row.dayHigh,dayLow: row.dayLow,closePrice:row.closePrice,open:row.open,timestamp_ID:row.timestamp_ID, stock_market_ID:row.stock_market_ID,
company_ID:row.company_ID }) return row.symbol;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/companyN.csv' as row FIELDTERMINATOR ',' CREATE (:company {company_ID:row.company_ID,symbol:row.symbol,companyName:row.companyName }) return row.company_ID;
 '''
,

'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/stock_market.csv' as row FIELDTERMINATOR ',' CREATE (:stock_market {ID:row.ID, country:row.country, stock_name:row.stock_name }) return row.country;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/time.csv' as row FIELDTERMINATOR ',' CREATE (:time {ID:row.ID, year:row.year, month:row.month, day:row.day, hour:row.hour, timestamp:row.timestamp}) return row.ID;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/tweets.csv' as row FIELDTERMINATOR ',' CREATE (:tweets {tweet_ID:row.tweet_ID,symbol:row.company_name,text:row.text,support_Count:row.support_Count,rank:row.rank,sentiment:row.sentiment,polarity:row.polarity,company_ID:row.company_ID,timestamp_ID:row.timestamp_ID}) return row.tweet_ID;
''',
# '''
# LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/tweets.csv' as row FIELDTERMINATOR ',' CREATE (:users {tweet_ID:row.tweet_ID, userID: row.userID}) return row.tweetID
# ''',
# '''
# LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/mentioned.csv' as row FIELDTERMINATOR ',' CREATE (:mentioned_users {tweet_ID:row.tweet_ID ,mentioned_userID:row.mentioned_userID}) return row.tweetID;
# ''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/user_data.csv' as row FIELDTERMINATOR ',' CREATE (:user_data {ID:row.ID, name:row.name ,followers_count:row.followers_count}) return row.ID;
'''
]


for i, nd_query in enumerate(node_queries):
    result = conn.query(nd_query, db=db_config.DATABASE)
    print("query {} executed \n {} \n".format(i+1,nd_query))


query 1 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/stock_data.csv' as row FIELDTERMINATOR ',' CREATE (:stock {symbol:row.symbol,lastPrice :row.lastPrice,
dayHigh: row.dayHigh,dayLow: row.dayLow,closePrice:row.closePrice,open:row.open,timestamp_ID:row.timestamp_ID, stock_market_ID:row.stock_market_ID,
company_ID:row.company_ID }) return row.symbol;
 

query 2 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/companyN.csv' as row FIELDTERMINATOR ',' CREATE (:company {company_ID:row.company_ID,symbol:row.symbol,companyName:row.companyName }) return row.company_ID;
  

query 3 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/stock_market.csv' as row FIELDTERMINATOR ',' CREATE (:stock_market {ID:row.ID, country:row.country, stock_name:row.stock_name }) return row.country;
 

query 4 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/time.csv' as row FIELDTERMINATOR ',' 

In [6]:


#make_relations
#-----------------------------------------------

relation_queries = [
'''
MATCH (st:stock),(company:company)
where st.company_ID = company.company_ID 
CREATE (st)-[r:belongs_to]->(company) 
return r
''',
'''
MATCH (t:tweets),(company:company)
where t.company_ID = company.company_ID 
CREATE (t)-[r:is_related_to]->(company) 
return r

''',
'''
MATCH (st:stock),(t:time)
where st.timestamp_ID = t.ID 
CREATE (st)-[r:at]->(t) 
return r

''',
'''
MATCH (t:tweets),(time: time)
where t.timestamp_ID = time.ID
CREATE (t)-[r:at_this]->(time) 
return r
 
''',
'''
MATCH (s:stock),(sm: stock_market)
where s.stock_market_ID = sm.ID
CREATE (s)-[r:is_in]->(sm) 
return r
'''
    #,    

    # '''
# MATCH (t:tweets),(tu: users)
# where t.tweet_ID = tu.tweet_ID
# CREATE (t)-[r:Postedby]->(tu) 
# return r

# '''
# ,    
# '''
# MATCH (t:tweets),(tu: mentioned_users)
# where t.tweet_ID = tu.tweet_ID
# CREATE (t)-[r:mentions]->(tu) 
# return r

# '''
# ,    
# '''
# MATCH (tu: users),(t:user_data)
# where tu.userID = t.ID
# CREATE (tu)-[r:user_detail]->(t) 
# return r

# '''
# ,    
# '''
# MATCH (tu: mentioned_users),(t:user_data)
# where tu.mentioned_userID = t.ID
# CREATE (tu)-[r:mentioned_userDetail]->(t) 
# return r

# '''
    
,
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/mentioned.csv' as row FIELDTERMINATOR ','
MATCH (t:user_data {ID: row.mentioned_userID}),(tu: tweets {tweet_ID: row.tweet_ID})
CREATE (tu)-[r:mentions]->(t) return r
'''
,
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/tweets.csv' as row FIELDTERMINATOR ','
MATCH (t:user_data {ID: row.userID}),(tu: tweets {tweet_ID: row.tweet_ID})
CREATE (tu)-[r:by_user]->(t) return r
'''
]
    

for j,re_query in enumerate(relation_queries):
    result = conn.query(re_query, db=db_config.DATABASE)
    print("query {} executed \n {} \n".format(j+1,re_query))

# delete whole database
### match (a) -[r] -> () delete a, r
### match (a) delete a

query 1 executed 
 
MATCH (st:stock),(company:company)
where st.company_ID = company.company_ID 
CREATE (st)-[r:belongs_to]->(company) 
return r
 

query 2 executed 
 
MATCH (t:tweets),(company:company)
where t.company_ID = company.company_ID 
CREATE (t)-[r:is_related_to]->(company) 
return r

 

query 3 executed 
 
MATCH (st:stock),(t:time)
where st.timestamp_ID = t.ID 
CREATE (st)-[r:at]->(t) 
return r

 

query 4 executed 
 
MATCH (t:tweets),(time: time)
where t.timestamp_ID = time.ID
CREATE (t)-[r:at_this]->(time) 
return r
 
 

query 5 executed 
 
MATCH (s:stock),(sm: stock_market)
where s.stock_market_ID = sm.ID
CREATE (s)-[r:is_in]->(sm) 
return r
 

query 6 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data_/mentioned.csv' as row FIELDTERMINATOR ','
MATCH (t:user_data {ID: row.mentioned_userID}),(tu: tweets {tweet_ID: row.tweet_ID})
CREATE (tu)-[r:mentions]->(t) return r
 

query 7 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Proje